In [47]:
import imaplib
import email
import yaml
import pandas as pd
from email.header import decode_header
from bs4 import BeautifulSoup
import re



In [48]:
with open('/Users/ankitjha/Downloads/credentials.yaml') as f:
 content = f.read()
my_credentials = yaml.load(content, Loader = yaml.FullLoader)
user, password = my_credentials['email_id'], my_credentials['password']


In [51]:
imap_url = "imap.gmail.com"
my_mail = imaplib.IMAP4_SSL(imap_url)
print(my_mail.login(user, password))

('OK', [b'ajha211@gmail.com authenticated (Success)'])


In [53]:
my_mail.select('hdfc_alerts')


('OK', [b'7'])

In [55]:
email_from="alerts@hdfcbank.net"
_, messages = my_mail.search(None, f'FROM "{email_from}"')

In [56]:
 # Convert the result list to a list of email IDs
email_ids = messages[0].split()

email_data = []

In [59]:
for i in email_ids[-100:]:
        # Fetch the email by ID
        res, msg = my_mail.fetch(i, "(RFC822)")

        for response_part in msg:
            if isinstance(response_part, tuple):
                # Parse the bytes email into a message object
                msg = email.message_from_bytes(response_part[1])
                
                # Decode email subject
                subject, encoding = decode_header(msg["Subject"])[0]
                if isinstance(subject, bytes):
                    # If it's a bytes type, decode to str
                    subject = subject.decode(encoding if encoding else "utf-8")
                
                # Decode email sender
                from_ = msg.get("From")
                
                # Extract the body of the email
                if msg.is_multipart():
                    # Iterate over email parts
                    for part in msg.walk():
                        # Extract content type of email
                        content_type = part.get_content_type()
                        content_disposition = str(part.get("Content-Disposition"))
                        
                        try:
                            # Get the email body
                            body = part.get_payload(decode=True).decode()
                        except:
                            pass
                else:
                    # Extract content type of email
                    content_type = msg.get_content_type()
                    
                    # Get the email body
                    body = msg.get_payload(decode=True).decode()

                email_data.append({
                    "subject": subject,
                    "from": from_,
                    "body": body,
                    "txn type": "",
                    "txn method": "",
                    "credit_debit":"",
                })

In [60]:
for i,j in df.iterrows():
    print(df['body'][i])

Dear Customer,  Rs.10.00 has been debited from account **5090 to VPA pahal.duggal@oksbi on 25-05-24. Your UPI transaction reference number is 414647378839
Dear Customer,Rs.184.00 has been debited from your HDFC Bank RuPay Credit Card XX7376 to starbucks.payu@icici on 25-05-24. Your UPI transaction reference number is 414654290754
Dear Customer,  Rs.320.00 has been debited from account **5090 to VPA pahal.duggal@oksbi on 25-05-24. Your UPI transaction reference number is 414662482789
Dear Customer,Rs.370.50 has been debited from your HDFC Bank RuPay Credit Card XX7376 to gedditconvenience.rzp@hdfcbank on 26-05-24. Your UPI transaction reference number is 414781192957
Dear Customer,Rs.387.85 has been debited from your HDFC Bank RuPay Credit Card XX7376 to paytm-51955531@paytm on 26-05-24. Your UPI transaction reference number is 414788948942


In [61]:
for email in email_data:
    soup = BeautifulSoup(email["body"], "html.parser")
    text = soup.get_text()
    pattern = r'Dear.*?reference number is \d+'
    # Use re.findall to find all matches
    matches = re.findall(pattern, text, re.DOTALL)
    email["body"]=matches[0]

    if "UPI" in email["subject"]:
        email["txn type"]="UPI"
        if "credit card" in email["body"]:
            email["txn method"]="CreditCard UPI"
    else:
        email["txn type"]="Other"


    if "debited from" in email["body"]:
        email["credit_debit"]="debit"

    if "credit" in email["body"]:
        email["credit_debit"]="credit"
    


       
    
    
   
    

In [62]:
df = pd.DataFrame(email_data)
df
   

,subject,from,body,txn type,txn method,credit_debit
0,❗ You have done a UPI txn. Check details!,HDFC Bank InstaAlerts <alerts@hdfcbank.net>,"Dear Customer, Rs.10.00 has been debited from...",UPI,,debit
1,❗ You have done a UPI txn. Check details!,HDFC Bank InstaAlerts <alerts@hdfcbank.net>,"Dear Customer,Rs.184.00 has been debited from ...",UPI,,debit
2,❗ You have done a UPI txn. Check details!,HDFC Bank InstaAlerts <alerts@hdfcbank.net>,"Dear Customer, Rs.320.00 has been debited fro...",UPI,,debit
3,❗ You have done a UPI txn. Check details!,HDFC Bank InstaAlerts <alerts@hdfcbank.net>,"Dear Customer,Rs.370.50 has been debited from ...",UPI,,debit
4,❗ You have done a UPI txn. Check details!,HDFC Bank InstaAlerts <alerts@hdfcbank.net>,"Dear Customer,Rs.387.85 has been debited from ...",UPI,,debit
5,❗ You have done a UPI txn. Check details!,HDFC Bank InstaAlerts <alerts@hdfcbank.net>,"Dear Customer,Rs.30.00 has been debited from y...",UPI,,debit
6,❗ You have done a UPI txn. Check details!,HDFC Bank InstaAlerts <alerts@hdfcbank.net>,"Dear Customer,Rs.2254.00 has been debited from...",UPI,,debit


### Email Parser


'Dear Customer,  Rs.320.00 has been debited from account **5090 to VPA pahal.duggal@oksbi on 25-05-24. Your UPI transaction reference number is 414662482789'